# DATA OPTIMIZER

## Global parameters

In [1]:
%cd '/UltimeTradingBot/Crypto_backtest_tools'

DATAFILE='/UltimeTradingBot/Data/w20_CryIn_NoVol.csv'

Normalization_File='/UltimeTradingBot/Crypto_backtest_tools/Data/w20_Noramalization_val_004.json'
NORMALIZATION_FILENAME=Normalization_File
Model_FileName='/UltimeTradingBot/Crypto_backtest_tools/Data/W20_004_Model.hdf5'
MODEL_FILENAME=Model_FileName
WINDOW_SIZE=20
window=WINDOW_SIZE
MAX_FORCAST_SIZE=10
BUY_PERCENT=0.4
SELL_PERCENT=2
DATA_DIR='/UltimeTradingBot/Data/'


/UltimeTradingBot/Crypto_backtest_tools


In [3]:
import sys
sys.path.append('/UltimeTradingBot/Crypto_backtest_tools')
%cd /UltimeTradingBot/Crypto_backtest_tools
from utilities.get_data import get_historical_from_db
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
#import ta
import numpy as np
import gc
import random
from sklearn.utils import shuffle
import seaborn as sns
import tensorflow as tf

from utilities.backtesting import plot_wallet_vs_asset, get_metrics, get_n_columns, basic_multi_asset_backtest, plot_sharpe_evolution, plot_bar_by_month
#from utilities.custom_indicators import SuperTrend
pd.options.mode.chained_assignment = None  # default='warn'
import gc
gc.collect()    

import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

PDEBUG=True
def pdebug(err):
    if PDEBUG:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

/UltimeTradingBot/Crypto_backtest_tools


2022-09-28 06:02:09.773683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 06:02:09.889925: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
df=pd.read_csv(DATAFILE)

In [13]:
df[(df["sell"]==1) & (df["bs"]==0)]

,index,price,high-1,low-1,open-1,close-1,high-2,low-2,open-2,close-2,...,BTC_open-20_5min,BTC_close-20_5min,BTC_volume-20_5min,day,hour,minute,lunch_day,buy,sell,bs
81,81,0.428233,-0.001323,0.002647,0.002647,-0.001323,-0.001323,0.002647,0.002647,-0.001323,...,0.001630,0.000510,122.173770,4,10,28,-667,0,1,0
102,102,6.561667,-0.010414,0.020066,0.013970,-0.002794,-0.010414,0.020066,0.013970,-0.002794,...,0.002674,0.001150,155.852040,6,2,9,-716,0,1,0
154,154,0.022617,-0.003685,0.005158,0.000737,0.005158,-0.003685,0.005158,0.000737,0.005158,...,0.001768,0.000029,112.652970,3,22,4,-691,0,1,0
157,157,0.485033,-0.002405,0.003574,0.003574,0.000069,-0.002405,0.003574,0.003574,0.000069,...,0.003613,-0.003894,677.842860,3,0,10,-457,0,1,0
262,262,2.213850,-0.004856,0.007430,-0.000294,0.007430,-0.004856,0.007430,-0.000294,0.007430,...,0.001255,-0.000637,85.499199,2,21,58,-273,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064623,1064623,0.035867,-0.003717,0.004647,0.004647,0.001859,-0.003717,0.004647,0.004647,0.001859,...,0.000448,-0.000355,165.900140,6,17,34,-691,0,1,0
1064703,1064703,0.372217,-0.002373,0.003806,0.003806,-0.000493,-0.002373,0.003806,0.003806,-0.000493,...,0.005498,0.000407,742.160271,5,15,47,-211,0,1,0
1064847,1064847,0.005869,-0.004998,0.008804,0.003862,0.002329,-0.004998,0.008804,0.003862,0.002329,...,0.005019,-0.000978,573.982200,5,16,14,-51,0,1,0
1064849,1064849,2.276383,-0.005235,0.007373,0.003287,0.005045,-0.005235,0.007373,0.003287,0.005045,...,0.003013,0.000028,168.196782,1,0,53,-223,0,1,0


In [14]:
df=df.drop(columns=["bs","sell","index"])

In [15]:
gc.collect()    


823

In [6]:
!rm '/UltimeTradingBot/Data/w20_CryIn_NoVol.csv'
df.to_csv("/UltimeTradingBot/Data/w20_CryIn_NoVol.csv")